In [ ]:
import pandas as pd
import numpy as np
import heapq
import math
import matplotlib.pyplot as plt

file_path = r'D:/KSU/SUMMER 2024/AI AND ROBOTICS/AI+R/grid.csv'
grid = pd.read_csv(file_path, header=None)

start_position = None
goal_position = None

for index, row in grid.iterrows():
    for col_index, value in enumerate(row):
        if value == 0:
            if start_position is None:
                start_position = (index, col_index)
            goal_position = (index, col_index)

print(f"Start Position: {start_position}")
print(f"Goal Position: {goal_position}")


def heuristic(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)


def line_of_sight(s, s_prime, grid):
    x0, y0 = s
    x1, y1 = s_prime
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy
    while (x0, y0) != (x1, y1):
        if grid.iloc[x0, y0] == 1:
            return False
        e2 = err * 2
        if e2 > -dy:
            err -= dy
            x0 += sx
        if e2 < dx:
            err += dx
            y0 += sy
    return True


def theta_star(grid, start, goal):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for dx, dy in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1]:
                if grid.iloc[neighbor[0], neighbor[1]] == 1:
                    continue

                if current in came_from:
                    parent = came_from[current]
                else:
                    parent = start

                if line_of_sight(parent, neighbor, grid):
                    tentative_g_score = g_score[parent] + heuristic(parent, neighbor)
                else:
                    tentative_g_score = g_score[current] + heuristic(current, neighbor)

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None


path = theta_star(grid, start_position, goal_position)
print(f"Path: {path}")


def plot_path(grid, path, start, goal):
    grid_numeric = grid.apply(pd.to_numeric, errors='coerce').fillna(1).astype(int)
    fig, ax = plt.subplots()
    ax.imshow(grid_numeric, cmap=plt.cm.gray)

    for (x, y) in path:
        grid_numeric.iloc[x, y] = 2

    ax.scatter(start[1], start[0], marker='o', color='green')
    ax.scatter(goal[1], goal[0], marker='x', color='red')
    ax.plot([y for x, y in path], [x for x, y in path], color='blue', linewidth=0.6)

    plt.show()


plot_path(grid, path, start_position, goal_position)
